# Quality Analysis of Cell Data

In [6]:
!pip install hana_ml
!pip install hdfs

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 41 kB 513 kB/s eta 0:00:01
  Created wheel for hdfs: filename=hdfs-2.5.8-py3-none-any.whl size=33213 sha256=6ba8f85a50d5098d957d20e9501d9dea0a4e776185a5e4775b2248dc196e5e52
  Stored in directory: /Users/d051079/Library/Caches/pip/wheels/0a/7d/38/ea4eaf831518e6cd867b515b88919a9785eb66f11def5ab859
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=c3f96baa0a6d86caede8f3cb33e2888796026bb65f7fbf804506dd79e0912d14
  Stored in directory: /Users/d051079/Library/Caches/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built hdfs docopt
You should consider upgrading via the '/Library/Developer/CommandLin

In [7]:
import hana_ml
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
from hdfs import InsecureClient

import pandas as pd
import matplotlib.pyplot as plt

## Connection to Data Source

In [8]:
#FOR HANA ML
conn = NotebookConnectionContext(connectionId = 'HANA_CLOUD_TECHED')
df = conn.table('TEADM_CELLSTATUS', schema='TECHED').collect()
display(df)

,DATE,CELLID,KEY1,KEY2,NOM_KEY1,NOM_KEY2
0,2020-11-01,1234512,98.964450,131.70491,74.4,147.4
1,2020-11-01,1345331,76.107155,203.95079,114.1,238.2
2,2020-11-01,4139991,123.091020,264.46765,102.1,122.6
3,2020-11-01,4543213,102.011925,125.23777,92.5,199.1
4,2020-11-01,7148927,104.337550,188.75677,109.5,192.4
...,...,...,...,...,...,...
175,2020-11-30,1345331,116.109810,0.00000,91.3,165.0
176,2020-11-30,4139991,132.113330,209.94498,85.3,156.7
177,2020-11-30,4543213,122.740650,203.76453,37.4,70.0
178,2020-11-30,7148927,86.377700,245.37593,115.9,203.4


## Configuration Setting and Performance over Time

In [5]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 5))
fig.autofmt_xdate(rotation=45)
#fig.tight_layout()

fig.suptitle('CELLSTATUS')
ax1.plot(df['DATE'],df['KEY1'])
ax1.plot(df['DATE'],df['NOM_KEY1'],color='red')
ax1.set(xlabel='DATE', ylabel='KEY1')
ax2.plot(df['DATE'],df['KEY2'])
ax2.plot(df['DATE'],df['NOM_KEY2'],color='red')
ax2.set(xlabel='DATE', ylabel='KEY2')

"\nfig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 5))\nfig.autofmt_xdate(rotation=45)\n#fig.tight_layout()\n\nfig.suptitle('CELLSTATUS')\nax1.plot(df['DATE'],df['KEY1'])\nax1.plot(df['DATE'],df['NOM_KEY1'],color='red')\nax1.set(xlabel='DATE', ylabel='KEY1')\nax2.plot(df['DATE'],df['KEY2'])\nax2.plot(df['DATE'],df['NOM_KEY2'],color='red')\nax2.set(xlabel='DATE', ylabel='KEY2')\n"

## Histogram of KEY1 and KEY2

In [4]:
plt.figure(figsize=(20,5))
plt.hist(df['KEY1'],50, facecolor='green', alpha=0.75)
plt.hist(df['KEY2'],50, facecolor='darkblue', alpha=0.75)

## Statistic Description

In [11]:
mean = df['KEY1'].mean()
std = df['KEY1'].std()
print('KEY1 mean: {}   std: {}'.format(mean,std))
exl_z3 = df.loc[ (df.KEY1 < mean - 3*std) | (df.KEY1 > mean + 3*std),'KEY1'].count() 
print('Z3-score: {}  ({})'.format(exl_z3,0.0027*df.shape[0]) )

mean = df['KEY2'].mean()
std = df['KEY2'].std()
print('KEY2 mean: {}   std: {}'.format(mean,std))
exl_z3 = df.loc[ (df.KEY2 < mean - 3*std) | (df.KEY2 > mean + 3*std),'KEY2'].count() 
print('Z3-score: {}  ({})'.format(exl_z3,0.0027*df.shape[0]) )

KEY1 mean: 104.22986389444446   std: 29.68762631165119
Z3-score: 1  (0.48600000000000004)
KEY2 mean: 197.97795743888886   std: 56.641270592616465
Z3-score: 3  (0.48600000000000004)


# Access to Performance Data

In [2]:
client = InsecureClient('http://datalake:50070')
with client.read('/shared/teadm/performance.csv', encoding = 'utf-8') as reader:
    df = pd.read_csv(reader, sep=",",names=['date','cellid','key1','key2'])
display(df)